In [1]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service as ChromeService
from webdriver_manager.chrome import ChromeDriverManager

In [2]:
from selenium.webdriver.firefox.service import Service as FirefoxService
from webdriver_manager.firefox import GeckoDriverManager

In [3]:
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
import pandas as pd

In [4]:
driver_manual = webdriver.Chrome()
driver_manual.get("https://www.youtube.com/watch?v=k9immzcwtHg")

2.2 Load driver bằng Driver Manager

In [8]:
service = ChromeService(ChromeDriverManager().install())
driver_auto = webdriver.Chrome(service= service)
print(driver_auto.service.path)


C:\Users\Admin\.wdm\drivers\chromedriver\win64\125.0.6422.78\chromedriver-win32/chromedriver.exe


### Lấy dữ liệu từ cafeF

https://s.cafef.vn/bao-cao-tai-chinh/fpt/incsta/2023/0/0/0/0/ket-qua-hoat-dong-kinh-doanh-.chn

https://s.cafef.vn/bao-cao-tai-chinh/fpt/bsheet/2023/0/0/0/ket-qua-hoat-dong-kinh-doanh-.chn



In [38]:
def get_elements_value(items, convert_numeric = True):
    _ls = []
    if convert_numeric == False:
        for x in items:
            _ls.append(x.text)
    else:
        for x in items:
            num = x.text.strip().replace(',', '')
            if num.isnumeric():
                _ls.append(int(num))
            else: 
                _ls.append(num)
    return _ls

def year_col_process(driver, col):
    x_path = f"//*[@id='tableContent']/tbody/tr/td[{col}]"
    rows = driver.find_elements(By.XPATH, x_path)
    return rows


In [41]:
ticker = 'fpt'
report_type = 'bs'

year = 2023
look_back = 5
from_year = 2023 - look_back + 1

data = {}
criteria_names = {}

runable = True
while(runable):

    url = f"https://s.cafef.vn/bao-cao-tai-chinh/{ticker}/bsheet/{year}/0/0/0/ket-qua-hoat-dong-kinh-doanh-.chn"
    print(url)
    driver_auto.get(url)
    driver_auto.implicitly_wait(5)


    if 'criteria' not in criteria_names:
        name_elements = year_col_process(driver_auto, 1)
        criteria_names['criteria'] = get_elements_value(name_elements, False)

    year_elements = driver_auto.find_elements(By.XPATH, "//*[@id ='tblGridData']/tbody/tr/td")
    index_cols = {}
    i = 1
    for item in year_elements:
        str_item = item.text.strip()
        if str_item.isnumeric():
            index_cols[str_item] = i
        i+=1
    index_cols = dict(sorted(index_cols.items(), reverse= True))

    y = 0
    for key in index_cols:
        col = index_cols[key]
        items = year_col_process(driver_auto, col)
        data[key] = get_elements_value(items, True)
        print(key)
        y = int(key)

        if y == from_year:
            runable = False
            break
    if runable == True:
        year = y - 1



data = dict(sorted(data.items())) #Sort dictionary 

data = criteria_names | data #Merge two dictionary into one

df = pd.DataFrame(data)
df

save_as = f"{ticker}_{report_type}.xlsx"
writer = pd.ExcelWriter(save_as, engine='xlsxwriter', engine_kwargs= {'options': {'string_to_numbers': True}})
df.to_excel(writer, sheet_name = report_type, index= False )
writer.close()


https://s.cafef.vn/bao-cao-tai-chinh/fpt/bsheet/2023/0/0/0/ket-qua-hoat-dong-kinh-doanh-.chn
2023
2022
2021
2020
https://s.cafef.vn/bao-cao-tai-chinh/fpt/bsheet/2019/0/0/0/ket-qua-hoat-dong-kinh-doanh-.chn
2019
